In [0]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os
from scipy.io import arff
import pandas as pd

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "/content/drive/My Drive/Images"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

#load the dataset
path = "/content/drive/My Drive/Data/h1b-case-2016.csv"
df = pd.read_csv(path, encoding='utf-8')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
#Data processing

#drop some columns
processed_data = df.drop(["CASE_NUMBER", "VISA_CLASS", "CASE_SUBMITTED", "DECISION_DATE", "EMPLOYER_CITY", "EMP_STATE_abb", "EMPLOYER_POSTAL_CODE", "EMPLOYER_COUNTRY", "EMPLOYER_PHONE", "AGENT_ATTORNEY_NAME", "AGENT_ATTORNEY_CITY", "AGENT_ATTORNEY_STATE", "JOB_TITLE", "SOC_NAME", "PREVAILING_WAGE", "PW_UNIT_OF_PAY", "WAGE_UNIT_OF_PAY", "WORKSITE_CITY", "WORKSITE_COUNTY", "Worksite_STATE_abb", "WORKSITE_POSTAL_CODE", "YEAR", "NAIC_CODE", "WAGE_RATE_OF_PAY", "EMP_STATE_full", "EMP_State_and_city", "Worksite_State_and_city"], axis=1)

#ignore the withdrawn cases
processed_data = processed_data[processed_data.CASE_STATUS != 'WITHDRAWN']
processed_data = processed_data[processed_data.CASE_STATUS != 'CERTIFIED-WITHDRAWN']

#concatenate the first 2 digit of column SOC_CODE
processed_data['SOC_CODE'] = processed_data['SOC_CODE'].map(lambda x: str(x)[:2])
processed_data['SOC_CODE'] = processed_data['SOC_CODE'].astype('int')

#Numericalize colums of CASE_STATUS and FULL_TIME_POSITION
Numericalize_columns = {"CASE_STATUS":{"CERTIFIED":0, "DENIED":1}, 
                        "FULL_TIME_POSITION":{"Y":1, "N":0}}
processed_data.replace(Numericalize_columns, inplace=True)

#Numericalize colums of Worksite_STATE_full
processed_data['Worksite_STATE_full'] = processed_data['Worksite_STATE_full'].astype('category')
#worksite_state_dict stores the mapping of numerical values to categorical values of Worksite_STATE_full, incase of further reference
worksite_state_dict = dict(zip(processed_data['Worksite_STATE_full'].cat.codes, processed_data['Worksite_STATE_full']))
processed_data['Worksite_STATE_full'] = processed_data['Worksite_STATE_full'].cat.codes.astype('int')

#Numericalize colums of EMPLOYMENT_START_DATE and EMPLOYMENT_END_DATE
processed_data['EMPLOYMENT_START_DATE'] = pd.to_datetime(processed_data['EMPLOYMENT_START_DATE'],infer_datetime_format=True).map(lambda x: str(x)[:4])
processed_data['EMPLOYMENT_END_DATE'] = pd.to_datetime(processed_data['EMPLOYMENT_END_DATE'],infer_datetime_format=True).map(lambda x: str(x)[:4])
processed_data = processed_data[processed_data.EMPLOYMENT_START_DATE != 'NaT']
processed_data = processed_data[processed_data.EMPLOYMENT_END_DATE != 'NaT']
processed_data['EMPLOYMENT_START_DATE'] = processed_data['EMPLOYMENT_START_DATE'].astype(int)
processed_data['EMPLOYMENT_END_DATE'] = processed_data['EMPLOYMENT_END_DATE'].astype(int)
processed_data['EMP_PERIOD'] = processed_data['EMPLOYMENT_END_DATE'] - processed_data['EMPLOYMENT_START_DATE']
processed_data = processed_data.drop(["EMPLOYMENT_START_DATE", "EMPLOYMENT_END_DATE"], axis=1)

#Numericalize colums of EMPLOYER_NAME
processed_data['EMPLOYER_NAME'] = processed_data['EMPLOYER_NAME'].astype('category')
#employer_name_dict stores the mapping of numerical values to categorical values of EMPLOYER_NAME, incase of further reference
employer_name_dict = dict(zip(processed_data['EMPLOYER_NAME'].cat.codes, processed_data['EMPLOYER_NAME']))
processed_data['EMPLOYER_NAME'] = processed_data['EMPLOYER_NAME'].cat.codes

In [169]:
processed_data.dtypes

CASE_STATUS              int64
EMPLOYER_NAME            int32
SOC_CODE                 int64
FULL_TIME_POSITION       int64
total_wage             float64
Sector_data              int64
Worksite_STATE_full      int64
EMP_PERIOD               int64
dtype: object